# .s2p Reader

In [1]:
import os
import numpy as np
import skrf as skrf
import cmath
import math as m

#Calcula los parámetros S en una frecuencia dada para cada archivo .s2p en las carpetas especificadas.
def calcular_parametros_s(frecuencia, carpetas):
    s_parameters = {}
    
    # Convertir frecuencia de GHz a Hz
    frecuencia *= 1e9                                  
    
    for carpeta in carpetas:
        archivos_s2p = [f for f in os.listdir(carpeta) if f.endswith('.s2p')]
        for archivo_s2p in archivos_s2p:
            ruta_archivo = os.path.join(carpeta, archivo_s2p)
            red = skrf.network.Network(ruta_archivo)
            
            # Encontrar la frecuencia más cercana en el archivo .s2p
            indice_frecuencia = np.argmin(np.abs(red.f - frecuencia))
                        
            # Acceder a los parámetros S en la frecuencia encontrada
            if archivo_s2p not in s_parameters:
                s_parameters[archivo_s2p] = red.s[indice_frecuencia]
    
    return s_parameters

matplotlib not found while setting up plotting


### Cálculo de Delta $\Delta$

In [2]:
def calculo_delta(parametros_s):
    dts= {}
    dts_abs= {}
    for archivo, parametros in parametros_s.items():
        s11= parametros[0, 0]
        s12= parametros[0, 1]
        s21= parametros[1, 0]
        s22= parametros[1, 1]

        # delta = (parametros[0, 0] * parametros[1, 1]) - (parametros[0, 1] * parametros[1, 0])
        dt= s11*s22 - s12*s21
        dts[archivo]= dt
        dts_abs[archivo]= abs(dt)

    return dts_abs

### Cálculo de Factor de Rollet $K$

In [3]:
def calcular_k(s_parametros, deltas):
    k = {}
    k_incond = {}
    for archivo, parametros in s_parametros.items():
        s11 = parametros[0, 0]
        s12 = parametros[0, 1]
        s21 = parametros[1, 0]
        s22 = parametros[1, 1]
        abs_delta = abs(deltas[archivo])
        
        # Calcular el parámetro K
        K= (1-abs(s11)**2 - abs(s22)**2 + (abs_delta)**2)/(2*abs(s12*s21))
        
        k[archivo]= K
        if K > 1:
            # Guardar la polarización si k es mayor que 1
            k_incond[archivo] = {'k': K}
    
    return k_incond


### Cálculo de $\Gamma_{in}, \Gamma_{out}$

In [4]:
def calcular_impedancias(Zo, s_parameters, polarizaciones_buenas_k, ic_filter, frecuencia):
    frecuencia *= 1e9  
    for nombre_archivo, polarizacion in polarizaciones_buenas_k.items():
        # Obtener el valor de IC del nombre del archivo
        ic  = float(nombre_archivo.split('_IC_') [1].split('mA')[0])
        vce = float(nombre_archivo.split('_VCE_')[1].split('V')[0])
        impedancias = []

        # Filtro de polarizaciones
        if ic < ic_filter:
            # Obtener los parámetros S de la polarización actual
            parametros_s = s_parameters[nombre_archivo]

            # Asignar los valores de los parámetros S a las variables correspondientes
            S11 = parametros_s[0, 0]
            S12 = parametros_s[0, 1]
            S21 = parametros_s[1, 0]
            S22 = parametros_s[1, 1]
            det = S11 * S22 - S12 * S21                 # Determinate

            # Cálculo de B1, B2, C1, C2
            B1 = 1 + abs(S11)**2 - abs(S22)**2 - abs(det)**2
            B2 = 1 + abs(S22)**2 - abs(S11)**2 - abs(det)**2
            C1 = S11 - det * np.conj(S22)
            C2 = S22 - det * np.conj(S11)
            
            # Cálculo de gamma in
            gamma_in = (B1 - np.sqrt((B1**2) - (4 * (abs(C1)**2)))) / (2 * abs(C1))
            gamma_in_rect = gamma_in * np.cos(np.angle(C1)) + gamma_in * np.sin(np.angle(C1)) * 1j

            # Cálculo de gamma out
            gamma_out = (B2 - np.sqrt((B2**2) - (4 * (abs(C2)**2)))) / (2 * abs(C2))
            gamma_out_rect = gamma_out * np.cos(np.angle(C2)) + gamma_out * np.sin(np.angle(C2)) * 1j

            "Adaptacion Entrada"
            # Cálculo impedancia de entrada serie
            Z_in = Zo * ((1 + gamma_in_rect) / (1 - gamma_in_rect))

            # Cálculo impedancia de entrada paralelo
            Z_in_p = np.real(Z_in) * (1+(np.imag(Z_in)/np.real(Z_in))**2) + 1j*np.imag(Z_in) * (1+(np.real(Z_in)/np.imag(Z_in))**2)

            # Cálculo de C_in adaptacion
            C_in =1/(2*np.pi*frecuencia*np.imag(Z_in_p))

            # Cálculo de transformador lamda entrada
            Z_in_trafo = np.sqrt(Zo*np.real(Z_in_p))

            
            "Adaptacion Salida"
            # Cálculo impedancia de salida serie
            Z_out = Zo * ((1 + gamma_out_rect) / (1 - gamma_out_rect)) 

            # Cálculo de transformador lamda salida
            Z_out_trafo = np.sqrt(Zo*np.real(Z_out))

            # Cálculo de L_out paralelo
            L_out_p = 1j*Z_out_trafo**2/np.imag(Z_out)

            # Cálculo de C_out adaptacion
            C_out =1/(2*np.pi*frecuencia*np.imag(-L_out_p))

               
            # Imprimir los resultados para cada configuración
            print(f"\033[1m\033[31mPolarización: Vce = {vce} [V] - Ic = {ic} [mA]\033[0m")
            print(f"S11: {S11:.4f}, S12: {S12:.4f}, S21: {S21:.4f}, S22: {S22:.4f}")
            print(f"B1 =", "{:.4f}".format(abs(B1)), "{:.4f}".format(np.degrees(np.angle(B1))), "°")
            print(f"B2 =", "{:.4f}".format(abs(B2)), "{:.4f}".format(np.degrees(np.angle(B2))), "°")
            print(f"C1 =", "{:.4f}".format(abs(C1)), "{:.4f}".format(np.degrees(np.angle(C1))), "°")
            print(f"C2 =", "{:.4f}".format(abs(C2)), "{:.4f}".format(np.degrees(np.angle(C2))), "°")
            print("Γᵢₙ      = ","{:.4f}".format(gamma_in), "{:.4f}".format(np.degrees(np.angle(C1))),"°")
            print("Γᵢₙ_rect = ","{:.4f}".format(gamma_in_rect))
            print("Γₒᵤ      = ","{:.4f}".format(gamma_out), "{:.4f}".format(np.degrees(np.angle(C2))),"°")
            print("Γₒᵤ_rect = ","{:.4f}".format(gamma_out_rect))
            print()
            print("ADAPTACION ENTRADA")
            print("Z_in     = ","{:.4f}".format(Z_in))
            print("C_in     = ","{:.4e}".format(C_in), "F") 
            print("Z_in_trafo= ","{:.4f}".format(Z_in_trafo))
            print()
            print("ADAPTACION SALIDA")
            print("Z_out    = ","{:.4f}".format(Z_out))  
            print("C_out    = ","{:.4e}".format(C_out), "F")
            print("Z_out_trafo= ","{:.4f}".format(Z_out_trafo))
            print()

            impedancias.append((Z_in, Z_out))
    
    return impedancias

### Diseño de microtiras

In [5]:
# def microtiras(Zo):
#     er = 4.3
#     A = Zo/60*np.sqrt((er+1)/2) + ((er-1)/(er+1))*(0.23 + (0.11/er))
#     B = 377*np.pi/(2*Zo*np.sqrt(er))

#     print("A: ", A, "B: ", B)


#     WH = (8*np.e**A)/(np.e**(2*A)-2)
#     print("WH: ", WH)

#     WH = 2*np.pi*(B-1-np.ln(2*B-1) + (er-1)/(2*er)*(np.ln(B-1) + 0.39*0.61/er))
#     print("WH: ", WH)

### Ejemplo de uso

In [6]:
frecuencia = 1.6                # GHz
Zo         = 50                 # Impedancia  
carpetas   = ["Datasheets/BFP640"]    # Rutas a las carpetas que contienen los archivos .s2p
ic_filter  = 40                # Corriente para filtrar polaricaciones

s_parameters = calcular_parametros_s(frecuencia, carpetas)
deltas= calculo_delta(s_parameters)
#k, k_incond= calcular_k(s_parameters, deltas)
k_incond= calcular_k(s_parameters, deltas)
impedancias  = calcular_impedancias(Zo, s_parameters, k_incond, ic_filter, frecuencia)


Polarización: Vce = 1.5 [V] - Ic = 37.0 [mA]
S11: -0.3731+0.0117j, S12: 0.0225+0.0361j, S21: 1.9106+11.5409j, S22: 0.1200-0.1726j
B1 = 0.9170 0.0000 °
B2 = 0.7266 0.0000 °
C1 = 0.4583 -178.2623 °
C2 = 0.3631 -47.2490 °
Γᵢₙ      =  0.9716 -178.2623 °
Γᵢₙ_rect =  -0.9711-0.0295j
Γₒᵤ      =  0.9643 -47.2490 °
Γₒᵤ_rect =  0.6545-0.7081j

ADAPTACION ENTRADA
Z_in     =  0.7211-0.7581j
C_in     =  -6.8882e-11 F
Z_in_trafo=  8.7125

ADAPTACION SALIDA
Z_out    =  5.6564-114.0767j
C_out    =  4.0123e-11 F
Z_out_trafo=  16.8172

Polarización: Vce = 1.0 [V] - Ic = 32.0 [mA]
S11: -0.4575+0.0416j, S12: 0.0275+0.0377j, S21: 1.8295+9.5005j, S22: 0.0484-0.1413j
B1 = 1.0347 0.0000 °
B2 = 0.6573 0.0000 °
C1 = 0.5090 178.5135 °
C2 = 0.3152 -52.3436 °
Γᵢₙ      =  0.8340 178.5135 °
Γᵢₙ_rect =  -0.8338+0.0216j
Γₒᵤ      =  0.7479 -52.3436 °
Γₒᵤ_rect =  0.4569-0.5921j

ADAPTACION ENTRADA
Z_in     =  4.5252+0.6433j
C_in     =  3.0632e-12 F
Z_in_trafo=  15.1932

ADAPTACION SALIDA
Z_out    =  34.1319-91.7221j
C_o

## Prints 

1) Todos los parámetros S para la frecuencia establecida (nuestro caso $1.6 \space GHz$)

In [7]:
# Parámetros S de todas las polarizaciones con frecuencia 1.6 GHz.
#print(s_parameters)

# Deltas calculados
#print(deltas)

# K que son incondicionalmente estables.
#print(k_incond)